# Test [U-Net](https://arxiv.org/abs/1505.04597)

Test a pretrained U-Net model on validation set.

In [1]:
# this is for path management in jupyter notebook
# note necessary if you're running in terminal or other IDEs
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Define the Network

In [2]:
# imports and parameter settings
import tensorflow as tf
from nn import unet
class_num = 2                 # class number in ground truth
patch_size = (572, 572)       # input patch size
lr = 1e-4                     # start learning rate
ds = 60                       # #epochs before lr decays
dr = 0.1                      # lr will decay to lr*dr
epochs = 1                    # #epochs to train
bs = 5                        # batch size
suffix = 'test'               # user customize name for the network

In [3]:
# define network
tf.reset_default_graph()
unet = unet.UNet(class_num, patch_size, suffix=suffix, learn_rate=lr, decay_step=ds, decay_rate=dr,
                 epochs=epochs, batch_size=bs)
overlap = unet.get_overlap()

## Prepare the Data

In [4]:
# imports and parameter settings
import numpy as np
from collection import collectionMaker, collectionEditor
ds_name = 'Inria'

In [5]:
cm = collectionMaker.read_collection(raw_data_path=r'/media/ei-edl01/data/uab_datasets/inria/data/Original_Tiles',
                                     field_name='austin,chicago,kitsap,tyrol-w,vienna',
                                     field_id=','.join(str(i) for i in range(37)),
                                     rgb_ext='RGB',
                                     gt_ext='GT',
                                     file_ext='tif',
                                     force_run=False,
                                     clc_name=ds_name)
gt_d255 = collectionEditor.SingleChanMult(cm.clc_dir, 1/255, ['GT', 'gt_d255']).\
    run(force_run=False, file_ext='png', d_type=np.uint8,)
cm.replace_channel(gt_d255.files, True, ['GT', 'gt_d255'])
cm.print_meta_data()
file_list_train = cm.load_files(field_id=','.join(str(i) for i in range(6, 37)), field_ext='RGB,gt_d255')
file_list_valid = cm.load_files(field_id=','.join(str(i) for i in range(6)), field_ext='RGB,gt_d255')
chan_mean = cm.meta_data['chan_mean'][:3]

gt_d255 might already exist, skip replacement
=========================================Inria==========================================
raw_data_path: /media/ei-edl01/data/uab_datasets/inria/data/Original_Tiles
field_name: ['austin', 'chicago', 'kitsap', 'tyrol-w', 'vienna']
field_id: [0, 1, ..., 36]
clc_name: Inria
tile_dim: (5000, 5000)
chan_mean: [ 103.2341876   108.95194825  100.14192002]
Source file: *RGB*.tif
GT file: *gt_d255*.png


## Test the Model

In [6]:
from nn import nn_utils
tile_size = (5000, 5000)
gpu = 0
sfn = 32

In [7]:
nn_utils.tf_warn_level(3) # Verbose information will not be displayed
model_dir = r'/hdd6/Models/test/unet_test_PS(572, 572)_BS5_EP6_LR0.0001_DS60_DR0.1'
tile_dict, field_dict, overall = unet.evaluate(file_list_valid, patch_size, tile_size, bs, chan_mean, model_dir, gpu, 
                                               save_result_parent_dir='ersa', sfn=sfn, force_run=False)

Summary of results:
===================austin=58.06===================
austin1=60.47
austin2=63.45
austin3=55.76
austin4=52.61
austin5=54.75
==================chicago=60.59===================
chicago1=60.98
chicago2=64.59
chicago3=59.27
chicago4=57.73
chicago5=57.33
===================kitsap=46.27===================
kitsap1=54.02
kitsap2=54.50
kitsap3=57.55
kitsap4=24.15
kitsap5=27.12
==================tyrol-w=52.88===================
tyrol-w1=52.91
tyrol-w2=51.78
tyrol-w3=54.09
tyrol-w4=50.72
tyrol-w5=54.09
===================vienna=68.81===================
vienna1=68.54
vienna2=73.19
vienna3=68.54
vienna4=69.62
vienna5=62.59
==================Overall=61.86===================
